# 自增数字 ID

作为通用的数据标识手段，数字 ID 对很多使用 Redis 的应用仍是不可获缺的。

有两种方案：
+ 使用字符串键
+ 使用哈希键

In [2]:
"""
配置连接
"""
from redis import Redis

# Redis连接配置
client = Redis(
    host='39.104.208.122', 
    port=6379,
    decode_responses=True,
    ssl=False
)

if client.ping():
    print("Redis连接成功")
else:
    print("Redis连接失败")

Redis连接成功


### 使用字符串键

逻辑就是，比如对于 `"UserID": "1"`，使用 `INCR key` 命令，就会变成`"UserID": "2"`。另外，如果这个 key 不存在，会自动初始化为 0，并加 1。

下面 `reverse()` 的目的就是定义一个初始值，`NX` 的目的是为了不重复创建这个初始值。

In [3]:
class IdGenerator:

    def __init__(self, client, name):
        self.client = client
        self.name = name

    def produce(self):
        """
        生成并返回下一个ID。
        """
        # INCR key
        return self.client.incr(self.name)

    def reserve(self, n):
        """
        尝试保留前N个ID，使得之后生成的ID都大于N。
        这个方法只能在执行produce()之前执行，否则函数将返回False表示执行失败。
        返回True则表示保留成功。
        """
        return self.client.set(self.name, n, nx=True) is True

In [5]:
gen = IdGenerator(client, "UserID")
gen.reserve(100)
print(gen.produce())

101


### 使用哈希键

与前者相比，使用哈希键实现的自增数字 ID 生成器可以将多个相关的 ID 生成器放到同一个键中进行管理，如 `"UserID_Coll": {"PostID": "1", "CommentID": "10"}`（Collection简写）。

In [7]:
class HashIdGenerator:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def produce(self, name):
        """
        生成并返回下一个ID。
        """
        # HINCRBY UserID_Coll CommentID 1
        return self.client.hincrby(self.key, name, 1)

    def reserve(self, name, number):
        """
        保留前N个ID，使得之后生成的ID都大于N。
        这个方法只能在执行produce()之前执行，否则函数将返回False表示执行失败。
        返回True则表示保留成功。
        """
        # HSETNX UserID_Coll CommentID 100
        return self.client.hsetnx(self.key, name, number) == 1